<a href="https://colab.research.google.com/github/IC-UFAL-IADD/IADD/blob/codebasics/%5B2022%5DIADD_FastAI_CNN_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Created by Marcelo Oliveira on 06/2021.
### Disciplina de Fundamentos de IA aplicados ao Diagnóstico de Doenças (IADD).
### Copyright (c)2021 Instituto de Computação
### Universidade Federal de Alagoas - UFAL. All rights reserved.

References:

https://www.kaggle.com/avirdee/siim-covid-19-initial-pipeline-fastai/notebook

In [9]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [10]:
!pip install python-gdcm pydicom -q
!pip install kornia -q
!pip install scikit-image -q

In [11]:
#import fastai; print(fastai.__version__)
#import fastcore; print(fastcore.__version__)
import pydicom,kornia,skimage

In [12]:
import pydicom,kornia,skimage
import os
import pandas as pd

In [13]:
from fastbook import *
from torch import nn

from fastai.callback.hook import summary
from fastai.callback.schedule import fit_one_cycle, lr_find 
from fastai.callback.progress import ProgressCallback

from fastai.data.core import Datasets, DataLoaders, show_at
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import Categorize, GrandparentSplitter, parent_label, ToTensor, IntToFloatTensor, Normalize

from fastai.layers import Flatten
from fastai.learner import Learner

from fastai.metrics import accuracy, CrossEntropyLossFlat

from fastai.vision.augment import CropPad, RandomCrop, PadMode
from fastai.vision.core import PILImageBW
from fastai.vision.utils import get_image_files

from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *
from fastai.medical.imaging import *


In [14]:
#Instalar antes de Montar Drive
!pip install umap-learn

     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=a07c49341c43cbfe4495def92cfe709d74f9dd22a8287832953c85ae905da682
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=9e5703c5b149805dbc093cec0ec6ea3718ff95d67fdcd359fd1337d021bf04dc
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [16]:
!touch "/content/drive/My Drive/ping.txt"
source=Path("/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/")
root="/content/drive/My Drive/DataSets/Classificacao_COVID_NORMAL/"
files = os.listdir(source)
print(files)

['COVID', 'normal', 'models', 'covid_normal.csv']


In [17]:
# A partir do das imagens gera o covid_normal.csv com a imagem(Path) e o label
os.chdir(root)

folders = ['normal', 'COVID']

files = []

for folder in folders:
    for file in os.listdir(folder):
        files.append([folder+'/'+file, folder])

pd.DataFrame(files, columns=['files', 'target']).to_csv(root+'covid_normal.csv')

In [18]:
df = pd.read_csv(source/'covid_normal.csv')
df.head()

,Unnamed: 0,files,target
0,0,normal/00e3a7e91a34.dcm,normal
1,1,normal/01a6995ec763.dcm,normal
2,2,normal/01aefe9ed777.dcm,normal
3,3,normal/01b3d67779e8.dcm,normal
4,4,normal/01bf68877dab.dcm,normal


In [19]:
class FixMonochrome(PILDicom):
    @classmethod
    def create(cls, fn:(Path, str, bytes), fix_monochrome = True)->None:
        if isinstance(fn, bytes): im = pydicom.dcmread(pydicom.filebase.DicomBytesIO(fn))
        if isinstance(fn, (Path, str)): im = pydicom.dcmread(fn)
        scaled = np.array(im.pixel_array)   
        if fix_monochrome and im.PhotometricInterpretation == "MONOCHROME1":
            scaled = np.amax(scaled) - scaled
        scaled = scaled - np.min(scaled)
        scaled = scaled / np.max(scaled)
        scaled = (scaled * 255).astype(np.uint8)
        pill_im = Image.fromarray(scaled)
        return cls(pill_im)

In [20]:
blocks = (ImageBlock(cls=FixMonochrome), CategoryBlock)


In [21]:
data_block = DataBlock(blocks=blocks,
                       getters = [ColReader('files', pref=source, suff=''), ColReader('target')],
                       item_tfms = [Resize(224, 224)],
                       splitter  = RandomSplitter(valid_pct=0.2, seed=42)
                      )

dls = data_block.dataloaders(df, bs=16, num_workers=0)  #bs (batch size), workers(processos na RAM, maior o bs maior o workers)

#dls.one_batch()

In [ ]:
learn = cnn_learner(dls, 
                    resnet34, 
                    metrics=[accuracy, error_rate])

In [ ]:
learn.fine_tune(4)

# Construir o Modelo

In [22]:
#ni: canal de entrada (ou nº de features de entrada)
#nf: features de saída
def conv(ni, nf): return nn.Conv2d(ni, nf, kernel_size=3, stride=2, padding=1)

In [23]:
# Batch Normalization: normalização dos dados para acelerar o treinamento;
def bn(nf): return nn.BatchNorm2d(nf)

In [24]:
def ReLU(): return nn.ReLU(inplace=False)

In [25]:
# Modelo
modelIADD = nn.Sequential(
    conv(1, 8),
    bn(8),
    ReLU(),
    conv(8, 16),
    bn(16),
    ReLU(),
    conv(16,32),
    bn(32),
    ReLU(),
    conv(32, 16),
    bn(16),
    ReLU(),
    conv(16, 10),
    bn(10),
    Flatten()
)

In [27]:
#xb, yb = first(dls.valid)
xb, yb = dls.one_batch()

In [28]:
#bs=16, chanel=1, size=224x224
xb.shape

torch.Size([16, 1, 224, 224])

In [29]:
# Total de CLasses
dls.c

2

In [30]:
# No método Learner o Dropout é ativado automaticamente
learn = Learner(
    dls, 
    modelIADD, 
    loss_func=F.cross_entropy, #funcao para minimizar o erro
    metrics=[accuracy, error_rate])

In [31]:
learn.summary()

Sequential (Input shape: 16 x 1 x 224 x 224)
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 8 x 112 x 112  
Conv2d                                    80         True      
BatchNorm2d                               16         True      
ReLU                                                           
____________________________________________________________________________
                     16 x 16 x 56 x 56   
Conv2d                                    1168       True      
BatchNorm2d                               32         True      
ReLU                                                           
____________________________________________________________________________
                     16 x 32 x 28 x 28   
Conv2d                                    4640       True      
BatchNorm2d                               64         True      
ReLU                                                           
___________________________________

Treinamento usando 4 épocas

In [36]:
learn.fine_tune(4)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,3.767224,4.541299,0.622951,0.377049,10:14


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,2.941144,3.103475,0.606557,0.393443,07:02
1,2.890906,2.349012,0.631148,0.368852,07:02
2,2.748677,2.378108,0.606557,0.393443,06:53
3,2.616279,2.314038,0.631148,0.368852,06:38


In [1]:
# Confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(title='Confusion matrix', figsize=(7,7))

NameError: ignored

In [ ]:
#Parametros da Matriz de Confusão
upp, low = interp.confusion_matrix()
VP, FN = upp[0], upp[1]
FP, VN = low[0], low[1]

print(VP, FN, FP, VN) #Dica: VP são as pessoas "Verdadeiramente Doentes", VN "Verdadeiramente NÃO Doentes"

### Sensibilidade e Especifidade (Sensitivity and Specificity)

In [ ]:
#Sensibilidade = Verdadeiro Positivo/(Verdadeiro Positivo + False Negativo)

Sensibilidade = VP/(VP + FN)
Sensibilidade


In [ ]:
#Especificidade = Verdadeiro Negativo/(False Positivo + Verdadeiro Negativo

Especificidade = VN/(FP + VN)
Especificidade

### Acuracia: Informa quantos de nossos exemplos foram de fato classificados corretamente, independente da classe. Ex.: Por exemplo, se temos 100 observações e 90 delas foram classificados corretamente, nosso modelo possui uma acurácia de 90%.

In [ ]:
preds,y, loss = learn.get_preds(with_loss=True)
# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc))

### AUC — Area Under the Curve ou AUROC — Area Under the Receiver Operating Characteristic curve): vantagem desta métrica é que ela não é sensível ao desbalanço de classes, como ocorre com a acurácia.

In [ ]:
from sklearn.metrics import roc_curve, auc
# probs from log preds
probs = np.exp(preds[:,1])
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y, probs, pos_label=1)

# Compute ROC area
roc_auc = auc(fpr, tpr)
print('ROC area is {0}'.format(roc_auc))

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")